# 括弧用2回目Finetune（fliplr=0.0 + Aidaデータ統合）

## このノートブックの目的

`prepare_paren_data.ipynb`で作成した括弧の擬似検出データと**Aidaデータを1:4の比率で混ぜて**、
**2回目のfinetune**を実行します。`fliplr=0.0`（左右反転なし）を設定することで、
`)` が `(` に吸われる問題を修正します。

### 処理の流れ
1. AWS認証設定
2. YOLOv5リポジトリのクローンとセットアップ
3. S3から括弧データとAidaデータをダウンロード・展開
4. Aidaデータから括弧クラス（42, 43）を除外
5. 各クラスから均等にサンプリング（新:旧 = 1:4）
6. データをtrain/valに分割（val_oldとval_newを別々に評価可能）
7. data.yamlの作成
8. hyp.yamlの作成（fliplr=0.0設定）
9. 既存のモデル重みからfinetuneを実行
10. 学習結果の確認
11. モデル重みをS3にアップロード

### 注意点
- **データ比率**: 新データ（括弧）: 旧データ（Aida） = 1:4
- **Aidaデータの括弧は除外**: Aidaデータの括弧クラス（42, 43）は使用しない（fliplr=0.0のため）
- **fliplr=0.0**: 左右反転を無効化（括弧の識別境界を矯正するため）
- **エポック数**: 5エポック（過学習を防ぐため）
- **学習率**: 0.001（既存モデルを壊さないため）
- **既存モデル**: 1回目のfinetuneで作成したモデルを使用


## セル0: AWS認証設定


In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "AKIAxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxxxxxxxxxxxxxxxxxx"  # 埋めてね！！！！！！！
os.environ["AWS_DEFAULT_REGION"] = "ap-northeast-1"

print("AWS env set")

assert "AWS_ACCESS_KEY_ID" in os.environ, "AWS_ACCESS_KEY_ID not set"
assert "AWS_SECRET_ACCESS_KEY" in os.environ, "AWS_SECRET_ACCESS_KEY not set"


## セル1: 設定（ここだけ触ればOK）


In [ ]:
# ============================================
# 設定（ここだけ変更すればOK）
# ============================================

import os
from pathlib import Path

# ===== S3設定 =====
S3_BUCKET = "km62m-ml-storage"  # S3バケット名
S3_PREFIX_PAREN = "yolo-dataset/finetune2-paren"  # 括弧データ
S3_PREFIX_AIDA = "yolo-dataset/v3"  # Aidaデータ

# ===== ローカルパス設定 =====
LOCAL_DATA_DIR = Path("/content/tmp/yolo_dataset_finetune2")  # データセットの展開先
LOCAL_OUTPUT_DIR = Path("/content/tmp/yolo_training_finetune2")  # 学習結果の出力先

# ===== 学習設定 =====
EPOCHS = 10  # エポック数（Aidaデータ統合により10に増加）
BATCH_SIZE = 16  # バッチサイズ（GPUメモリに応じて調整）
IMG_SIZE = 640  # 画像サイズ
DEVICE = 0  # GPU使用（0: GPU, 'cpu': CPU）

# ===== データ統合設定 =====
# 新データ（括弧）: 旧データ（Aida）の比率（推奨: 1:3 〜 1:5）
DATA_RATIO_NEW_TO_OLD = 1 / 4  # 1:4の比率（新:旧）
CLASS_ID_LPAREN = 42  # 左括弧のクラスID
CLASS_ID_RPAREN = 43  # 右括弧のクラスID
# Aidaデータから各クラス最大何枚まで使用するか（None=全量使用）
MAX_SAMPLES_PER_CLASS_AIDA = None  # Noneまたは数値（例: 2000）

# ===== train/val分割設定 =====
SEED = 42  # 再現性のためのシード
VAL_RATIO = 0.2  # 20%をvalに

# ===== モデル設定 =====
MODEL_NAME = "yolov5n"  # YOLOv5nモデルを使用
# 1回目のfinetuneで作成したモデル重み（S3から取得）
PRETRAINED_WEIGHTS_S3 = f"s3://{S3_BUCKET}/yolo-dataset/v3/weights/yolov5n_train_20251229_161158/best.pt"  # 実際のパスに置き換える
PRETRAINED_WEIGHTS_LOCAL = Path("/content/tmp/pretrained_weights/best.pt")  # ローカル保存先

# ===== その他 =====
CLEAN_LOCAL_DATA = False  # Trueにすると既存のローカルデータを削除

print("✓ 設定完了")
print(f"  S3_BUCKET: {S3_BUCKET}")
print(f"  S3_PREFIX_PAREN: {S3_PREFIX_PAREN}")
print(f"  S3_PREFIX_AIDA: {S3_PREFIX_AIDA}")
print(f"  LOCAL_DATA_DIR: {LOCAL_DATA_DIR}")
print(f"  EPOCHS: {EPOCHS}")
print(f"  BATCH_SIZE: {BATCH_SIZE}")
print(f"  IMG_SIZE: {IMG_SIZE}")
print(f"  VAL_RATIO: {VAL_RATIO}")
print(f"  DATA_RATIO_NEW_TO_OLD: 1:{int(1/DATA_RATIO_NEW_TO_OLD)} (新:旧)")
print(f"  MAX_SAMPLES_PER_CLASS_AIDA: {MAX_SAMPLES_PER_CLASS_AIDA or '全量'}")
print(f"  PRETRAINED_WEIGHTS_S3: {PRETRAINED_WEIGHTS_S3}")
print(f"\n⭐ 括弧データとAidaデータを1:{int(1/DATA_RATIO_NEW_TO_OLD)}の比率で統合します")
print(f"  ⚠️  Aidaデータの括弧クラス（{CLASS_ID_LPAREN}, {CLASS_ID_RPAREN}）は除外します")


## セル2: YOLOv5リポジトリのセットアップ


In [ ]:
# ============================================
# YOLOv5リポジトリのクローンと依存パッケージのインストール
# ============================================

import subprocess
import sys
from pathlib import Path

print("=" * 80)
print("YOLOv5リポジトリのセットアップ")
print("=" * 80)

# YOLOv5リポジトリのパス
YOLOV5_DIR = Path("/content/yolov5")

# YOLOv5リポジトリをクローン（既に存在する場合はスキップ）
if YOLOV5_DIR.exists() and (YOLOV5_DIR / "train.py").exists():
    print(f"\n✓ YOLOv5リポジトリは既にクローン済み: {YOLOV5_DIR}")
else:
    print(f"\n📦 YOLOv5リポジトリをクローン中...")
    print("   URL: https://github.com/ultralytics/yolov5")
    
    # 既存のディレクトリがあれば削除
    if YOLOV5_DIR.exists():
        import shutil
        shutil.rmtree(YOLOV5_DIR)
    
    subprocess.check_call([
        "git", "clone", 
        "https://github.com/ultralytics/yolov5.git",
        str(YOLOV5_DIR)
    ])
    print(f"✓ YOLOv5リポジトリのクローン完了")

# YOLOv5の依存パッケージをインストール
print(f"\n📦 YOLOv5の依存パッケージをインストール中...")
requirements_file = YOLOV5_DIR / "requirements.txt"
if requirements_file.exists():
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "-q", 
        "-r", str(requirements_file)
    ])
    print(f"✓ YOLOv5の依存パッケージのインストール完了")
else:
    print(f"⚠️  警告: requirements.txtが見つかりません")

# AWS CLI
try:
    result = subprocess.run(["aws", "--version"], capture_output=True, text=True)
    print(f"\n✓ AWS CLI は既にインストール済み: {result.stdout.strip()}")
except FileNotFoundError:
    print("\n📦 AWS CLI をインストール中...")
    subprocess.check_call(["pip", "install", "-q", "awscli"])
    print("✓ AWS CLI インストール完了")

print(f"\n✓ YOLOv5のセットアップ完了")
print(f"  YOLOv5ディレクトリ: {YOLOV5_DIR}")
print(f"  train.py: {YOLOV5_DIR / 'train.py'}")


## セル3: S3から括弧データとAidaデータをダウンロード・統合・train/val分割


In [ ]:
# ============================================
# S3から括弧データとAidaデータをダウンロード・統合・train/val分割
# ============================================

import subprocess
import tarfile
import shutil
import random
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict

print("=" * 80)
print("S3から括弧データとAidaデータをダウンロード・統合・train/val分割")
print("=" * 80)
print(f"⭐ データ比率: 新（括弧）: 旧（Aida） = 1:{int(1/DATA_RATIO_NEW_TO_OLD)}")
print(f"⚠️  Aidaデータの括弧クラス（{CLASS_ID_LPAREN}, {CLASS_ID_RPAREN}）は除外します")

# 既存データのクリーンアップ
if CLEAN_LOCAL_DATA and LOCAL_DATA_DIR.exists():
    print(f"\n🗑️  既存のローカルデータを削除: {LOCAL_DATA_DIR}")
    shutil.rmtree(LOCAL_DATA_DIR)

# ダウンロード先ディレクトリ
download_dir = LOCAL_DATA_DIR.parent / "yolo_tars_download_finetune2"
download_dir.mkdir(parents=True, exist_ok=True)

# データセットのディレクトリ構造を作成
LOCAL_DATA_DIR.mkdir(parents=True, exist_ok=True)
train_images_dir = LOCAL_DATA_DIR / "train" / "images"
train_labels_dir = LOCAL_DATA_DIR / "train" / "labels"
val_images_dir = LOCAL_DATA_DIR / "val" / "images"
val_labels_dir = LOCAL_DATA_DIR / "val" / "labels"

for d in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

# ===== 括弧データをダウンロード =====
print(f"\n📥 括弧データをダウンロード中...")
paren_tar_files = [
    "paren_train_images.tar",
    "paren_train_labels.tar"
]

for tar_name in paren_tar_files:
    s3_path = f"s3://{S3_BUCKET}/{S3_PREFIX_PAREN}/{tar_name}"
    local_tar_path = download_dir / tar_name
    
    if local_tar_path.exists():
        size_mb = local_tar_path.stat().st_size / (1024 * 1024)
        print(f"  ✓ {tar_name} は既にダウンロード済み: {size_mb:.2f} MB")
    else:
        print(f"  📥 {tar_name} をダウンロード中...")
        try:
            subprocess.run(
                ["aws", "s3", "cp", s3_path, str(local_tar_path)],
                check=True, capture_output=True
            )
            size_mb = local_tar_path.stat().st_size / (1024 * 1024)
            print(f"    ✓ 完了: {size_mb:.2f} MB")
        except subprocess.CalledProcessError as e:
            print(f"    ❌ 失敗: {e}")
            raise

# ===== Aidaデータをダウンロード =====
print(f"\n📥 Aidaデータをダウンロード中...")
aida_tar_files = [
    "train_images.tar",
    "train_labels.tar"
]

for tar_name in aida_tar_files:
    s3_path = f"s3://{S3_BUCKET}/{S3_PREFIX_AIDA}/{tar_name}"
    local_tar_path = download_dir / tar_name
    
    if local_tar_path.exists():
        size_mb = local_tar_path.stat().st_size / (1024 * 1024)
        print(f"  ✓ {tar_name} は既にダウンロード済み: {size_mb:.2f} MB")
    else:
        print(f"  📥 {tar_name} をダウンロード中...")
        try:
            subprocess.run(
                ["aws", "s3", "cp", s3_path, str(local_tar_path)],
                check=True, capture_output=True
            )
            size_mb = local_tar_path.stat().st_size / (1024 * 1024)
            print(f"    ✓ 完了: {size_mb:.2f} MB")
        except subprocess.CalledProcessError as e:
            print(f"    ❌ 失敗: {e}")
            raise

# ===== 括弧データを展開 =====
print(f"\n📦 括弧データを展開中...")
temp_paren_dir = LOCAL_DATA_DIR.parent / "temp_paren"
temp_paren_dir.mkdir(parents=True, exist_ok=True)

for tar_name in paren_tar_files:
    tar_path = download_dir / tar_name
    if tar_path.exists():
        target_dir = temp_paren_dir / tar_name.replace(".tar", "").replace("paren_", "")
        target_dir.mkdir(parents=True, exist_ok=True)
        with tarfile.open(tar_path, 'r') as tar:
            tar.extractall(target_dir)
        print(f"  ✓ {tar_name} 展開完了")

# ===== Aidaデータを展開 =====
print(f"\n📦 Aidaデータを展開中...")
temp_aida_dir = LOCAL_DATA_DIR.parent / "temp_aida"
temp_aida_dir.mkdir(parents=True, exist_ok=True)

for tar_name in aida_tar_files:
    tar_path = download_dir / tar_name
    if tar_path.exists():
        target_dir = temp_aida_dir / tar_name.replace(".tar", "")
        target_dir.mkdir(parents=True, exist_ok=True)
        with tarfile.open(tar_path, 'r') as tar:
            tar.extractall(target_dir)
        print(f"  ✓ {tar_name} 展開完了")

# ===== 括弧データの読み込み =====
print(f"\n📊 括弧データを読み込み中...")
paren_images = list((temp_paren_dir / "train_images").glob("*"))
paren_labels = list((temp_paren_dir / "train_labels").glob("*.txt"))

paren_image_uuids = {f.stem for f in paren_images}
paren_label_uuids = {f.stem for f in paren_labels}
paren_matched_uuids = sorted(list(paren_image_uuids & paren_label_uuids))

print(f"  括弧データ総数: {len(paren_matched_uuids)} 件")

# ===== Aidaデータの読み込み（括弧クラスを除外） =====
print(f"\n📊 Aidaデータを読み込み中（括弧クラス除外）...")
aida_images = list((temp_aida_dir / "train_images").glob("*"))
aida_labels = list((temp_aida_dir / "train_labels").glob("*.txt"))

aida_image_uuids = {f.stem for f in aida_images}
aida_label_uuids = {f.stem for f in aida_labels}
aida_matched_uuids = sorted(list(aida_image_uuids & aida_label_uuids))

print(f"  Aidaデータ総数（マッチング前）: {len(aida_matched_uuids)} 件")

# 括弧クラスを含むラベルを除外
aida_filtered_uuids = []
aida_class_counts = defaultdict(int)
excluded_count = 0

for uuid in tqdm(aida_matched_uuids, desc="Aidaデータフィルタリング"):
    label_file = temp_aida_dir / "train_labels" / f"{uuid}.txt"
    if not label_file.exists():
        continue
    
    # ラベルファイルを読み込んで括弧クラスが含まれているかチェック
    has_paren = False
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                aida_class_counts[class_id] += 1
                if class_id == CLASS_ID_LPAREN or class_id == CLASS_ID_RPAREN:
                    has_paren = True
                    break
    
    if not has_paren:
        aida_filtered_uuids.append(uuid)
    else:
        excluded_count += 1

print(f"  括弧クラスを含むデータを除外: {excluded_count} 件")
print(f"  Aidaデータ（括弧除外後）: {len(aida_filtered_uuids)} 件")

# クラス分布を表示
print(f"\n  Aidaデータのクラス分布（上位10クラス）:")
sorted_classes = sorted(aida_class_counts.items(), key=lambda x: x[1], reverse=True)
for class_id, count in sorted_classes[:10]:
    class_name = f"class_{class_id}"
    if class_id == CLASS_ID_LPAREN:
        class_name = "("
    elif class_id == CLASS_ID_RPAREN:
        class_name = ")"
    print(f"    クラス{class_id} ({class_name}): {count} 件")

# ===== 各クラスから均等にサンプリング（Aidaデータ） =====
print(f"\n📊 Aidaデータをクラスごとにサンプリング中...")
aida_by_class = defaultdict(list)

for uuid in aida_filtered_uuids:
    label_file = temp_aida_dir / "train_labels" / f"{uuid}.txt"
    if not label_file.exists():
        continue
    
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if class_id != CLASS_ID_LPAREN and class_id != CLASS_ID_RPAREN:
                    aida_by_class[class_id].append(uuid)
                    break  # 1つのラベルファイルに1つのクラスIDのみ

# 各クラスから最大MAX_SAMPLES_PER_CLASS_AIDA枚までサンプリング
random.seed(SEED)
aida_sampled_uuids = []
for class_id, uuids in aida_by_class.items():
    if MAX_SAMPLES_PER_CLASS_AIDA is not None:
        sampled = random.sample(uuids, min(len(uuids), MAX_SAMPLES_PER_CLASS_AIDA))
    else:
        sampled = uuids
    aida_sampled_uuids.extend(sampled)

# 重複を除去（1つのUUIDが複数のクラスに含まれる可能性があるため）
aida_sampled_uuids = list(set(aida_sampled_uuids))
random.shuffle(aida_sampled_uuids)

print(f"  サンプリング後: {len(aida_sampled_uuids)} 件")

# ===== 比率に合わせてAidaデータを調整 =====
num_paren = len(paren_matched_uuids)
target_aida = int(num_paren / DATA_RATIO_NEW_TO_OLD)

if len(aida_sampled_uuids) > target_aida:
    aida_sampled_uuids = aida_sampled_uuids[:target_aida]
    print(f"  比率調整後: {len(aida_sampled_uuids)} 件（目標: {target_aida} 件）")
else:
    print(f"  全量使用: {len(aida_sampled_uuids)} 件（目標: {target_aida} 件）")

print(f"\n  最終データ数:")
print(f"    括弧データ: {num_paren} 件")
print(f"    Aidaデータ: {len(aida_sampled_uuids)} 件")
print(f"    合計: {num_paren + len(aida_sampled_uuids)} 件")
print(f"    比率: 1:{len(aida_sampled_uuids)/num_paren:.2f} (新:旧)")

# ===== train/val分割 =====
print(f"\n📊 train/val分割中...")

# 括弧データを分割
random.seed(SEED)
random.shuffle(paren_matched_uuids)
paren_split_idx = int(len(paren_matched_uuids) * (1 - VAL_RATIO))
paren_train_uuids = paren_matched_uuids[:paren_split_idx]
paren_val_uuids = paren_matched_uuids[paren_split_idx:]

# Aidaデータを分割
random.seed(SEED)
random.shuffle(aida_sampled_uuids)
aida_split_idx = int(len(aida_sampled_uuids) * (1 - VAL_RATIO))
aida_train_uuids = aida_sampled_uuids[:aida_split_idx]
aida_val_uuids = aida_sampled_uuids[aida_split_idx:]

print(f"  括弧データ - Train: {len(paren_train_uuids)} 件, Val: {len(paren_val_uuids)} 件")
print(f"  Aidaデータ - Train: {len(aida_train_uuids)} 件, Val: {len(aida_val_uuids)} 件")

# ===== ファイルをコピー =====
print(f"\n📁 ファイルをコピー中...")

# 括弧データ（train）
for uuid in tqdm(paren_train_uuids, desc="括弧train"):
    img_files = list((temp_paren_dir / "train_images").glob(f"{uuid}.*"))
    if img_files:
        shutil.copy2(img_files[0], train_images_dir / img_files[0].name)
    label_file = temp_paren_dir / "train_labels" / f"{uuid}.txt"
    if label_file.exists():
        shutil.copy2(label_file, train_labels_dir / f"{uuid}.txt")

# 括弧データ（val）
for uuid in tqdm(paren_val_uuids, desc="括弧val"):
    img_files = list((temp_paren_dir / "train_images").glob(f"{uuid}.*"))
    if img_files:
        shutil.copy2(img_files[0], val_images_dir / img_files[0].name)
    label_file = temp_paren_dir / "train_labels" / f"{uuid}.txt"
    if label_file.exists():
        shutil.copy2(label_file, val_labels_dir / f"{uuid}.txt")

# Aidaデータ（train）
for uuid in tqdm(aida_train_uuids, desc="Aida train"):
    img_files = list((temp_aida_dir / "train_images").glob(f"{uuid}.*"))
    if img_files:
        shutil.copy2(img_files[0], train_images_dir / img_files[0].name)
    label_file = temp_aida_dir / "train_labels" / f"{uuid}.txt"
    if label_file.exists():
        shutil.copy2(label_file, train_labels_dir / f"{uuid}.txt")

# Aidaデータ（val）
for uuid in tqdm(aida_val_uuids, desc="Aida val"):
    img_files = list((temp_aida_dir / "train_images").glob(f"{uuid}.*"))
    if img_files:
        shutil.copy2(img_files[0], val_images_dir / img_files[0].name)
    label_file = temp_aida_dir / "train_labels" / f"{uuid}.txt"
    if label_file.exists():
        shutil.copy2(label_file, val_labels_dir / f"{uuid}.txt")

# ===== 結果の確認 =====
train_images = list(train_images_dir.glob("*"))
train_labels = list(train_labels_dir.glob("*.txt"))
val_images = list(val_images_dir.glob("*"))
val_labels = list(val_labels_dir.glob("*.txt"))

print(f"\n" + "=" * 80)
print("データセット準備完了")
print("=" * 80)
print(f"  Train画像: {len(train_images)} 件")
print(f"  Trainラベル: {len(train_labels)} 件")
print(f"  Val画像: {len(val_images)} 件")
print(f"  Valラベル: {len(val_labels)} 件")
print(f"\n✓ データ統合完了（括弧 + Aida、比率1:{int(1/DATA_RATIO_NEW_TO_OLD)}）")


## セル4: data.yamlとhyp.yamlの作成


In [ ]:
# ============================================
# data.yamlとhyp.yamlの作成
# ============================================

import yaml
import json
from pathlib import Path
from datetime import datetime

print("=" * 80)
print("data.yamlとhyp.yaml作成")
print("=" * 80)

# クラス定義（既存のYOLOモデルと一致）
# python/data/config/classes.pyから取得
target_classes = [
    # 数字（10クラス）
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    # 小数点（1クラス）
    '.',
    # 基本演算子（5クラス）
    '+', '-', '*', '/', '=',
    # アルファベット変数（小文字26クラス）
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    # 括弧類（3クラス）
    '(', ')', '|',
    # ギリシャ文字・定数（3クラス）
    'π', 'θ', '∞',
    # 特殊記号（2クラス）
    '→', '√',
    # 関数名（12クラス）
    'sin', 'cos', 'tan', 'sec', 'csc', 'cot', 'ln', 'log', 'exp', 'sqrt', 'lim', 'abs',
]

print(f"  クラス数: {len(target_classes)} クラス")
print(f"  括弧クラスID: '('={target_classes.index('(')}, ')'={target_classes.index(')')}")

# data.yamlを作成
data_config = {
    'path': str(LOCAL_DATA_DIR.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(target_classes),
    'names': {idx: cls for idx, cls in enumerate(target_classes)}
}

data_yaml_file = LOCAL_DATA_DIR / "data.yaml"
with open(data_yaml_file, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)

print(f"\n✓ data.yamlを作成: {data_yaml_file}")

# hyp.yamlを作成（fliplr=0.0設定）
# YOLOv5のデフォルトのhyp.yamlをベースに、fliplr=0.0を設定
hyp_config = {
    'lr0': 0.001,  # 初期学習率（既存モデルを壊さないため0.001に変更）
    'lrf': 0.1,   # 最終学習率（lr0 * lrf）
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'cls_pw': 1.0,
    'obj': 1.0,
    'obj_pw': 1.0,
    'iou_t': 0.20,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,  # 画像HSV-Hue augmentation
    'hsv_s': 0.7,    # 画像HSV-Saturation augmentation
    'hsv_v': 0.4,    # 画像HSV-Value augmentation
    'degrees': 0.0,  # 画像回転（+/-度）
    'translate': 0.1,  # 画像平行移動（+/- fraction）
    'scale': 0.5,     # 画像スケール（+/- gain）
    'shear': 0.0,     # 画像せん断（+/-度）
    'perspective': 0.0,  # 画像遠近法（+/- fraction）
    'flipud': 0.0,    # 画像上下反転（確率）
    'fliplr': 0.0,    # ⭐ 画像左右反転（確率）← 0.0に設定！
    'mosaic': 1.0,    # 画像モザイク（確率）
    'mixup': 0.0,     # 画像ミックスアップ（確率）
    'copy_paste': 0.0  # セグメンテーションコピー＆ペースト（確率）
}

hyp_yaml_file = LOCAL_DATA_DIR / "hyp.yaml"
with open(hyp_yaml_file, 'w', encoding='utf-8') as f:
    yaml.dump(hyp_config, f, default_flow_style=False, allow_unicode=True)

print(f"\n✓ hyp.yamlを作成: {hyp_yaml_file}")
print(f"  ⭐ fliplr=0.0 を設定しました（左右反転なし）")

print(f"\n✓ 設定ファイル作成完了")
data_path = str(data_yaml_file)
hyp_path = str(hyp_yaml_file)


## セル5: 事前学習済みモデルのダウンロード


In [ ]:
# ============================================
# 事前学習済みモデルのダウンロード
# ============================================

import subprocess

print("=" * 80)
print("事前学習済みモデルのダウンロード")
print("=" * 80)

# ローカル保存先の準備
PRETRAINED_WEIGHTS_LOCAL.parent.mkdir(parents=True, exist_ok=True)

if PRETRAINED_WEIGHTS_LOCAL.exists():
    size_mb = PRETRAINED_WEIGHTS_LOCAL.stat().st_size / (1024 * 1024)
    print(f"\n✓ 事前学習済みモデルは既にダウンロード済み: {size_mb:.2f} MB")
else:
    print(f"\n📥 事前学習済みモデルをダウンロード中...")
    print(f"  S3パス: {PRETRAINED_WEIGHTS_S3}")
    try:
        subprocess.run(
            ["aws", "s3", "cp", PRETRAINED_WEIGHTS_S3, str(PRETRAINED_WEIGHTS_LOCAL)],
            check=True
        )
        size_mb = PRETRAINED_WEIGHTS_LOCAL.stat().st_size / (1024 * 1024)
        print(f"  ✓ 完了: {size_mb:.2f} MB")
    except subprocess.CalledProcessError as e:
        print(f"  ❌ 失敗: {e}")
        print(f"  ⚠️  注意: PRETRAINED_WEIGHTS_S3のパスを確認してください")
        raise

pretrained_weights_path = str(PRETRAINED_WEIGHTS_LOCAL)
print(f"\n✓ 事前学習済みモデルの準備完了")
print(f"  パス: {pretrained_weights_path}")


## セル6: Finetuneの実行

**注意**: このセルは長時間かかります。GPUランタイムを使用してください。


In [ ]:
# ============================================
# Finetuneの実行（YOLOv5 train.py）※進捗をリアルタイム表示
# ============================================

import os
import subprocess
import sys
from pathlib import Path
from datetime import datetime

# ✅ wandbの対話待ちを無効化（30秒タイムアウト回避）
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_SILENT"] = "true"

print("=" * 80)
print("Finetune開始（YOLOv5 train.py）")
print("=" * 80)
print(f"開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# YOLOv5のtrain.pyのパス
train_script = YOLOV5_DIR / "train.py"
if not train_script.exists():
    raise FileNotFoundError(f"train.pyが見つかりません: {train_script}")

# 出力先の設定
LOCAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"{MODEL_NAME}_finetune2_{timestamp}"

# train.pyの実行コマンドを構築
train_cmd = [
    sys.executable,
    str(train_script),
    "--data", data_path,
    "--hyp", hyp_path,  # ⭐ hyp.yamlを指定（fliplr=0.0）
    "--weights", pretrained_weights_path,  # 事前学習済み重み
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--img", str(IMG_SIZE),
    "--device", str(DEVICE),
    "--project", str(LOCAL_OUTPUT_DIR),
    "--name", run_name,
    "--save-period", "5",
]

print("実行コマンド:")
print(f"  {' '.join(train_cmd)}")
print()

# 作業ディレクトリをYOLOv5に変更して実行
original_cwd = Path.cwd()
try:
    os.chdir(YOLOV5_DIR)
    print(f"作業ディレクトリ: {YOLOV5_DIR}")
    print()

    # ✅ リアルタイム表示で起動
    process = subprocess.Popen(
        train_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True,
    )

    # 進捗ログをリアルタイム表示
    assert process.stdout is not None
    for line in process.stdout:
        print(line, end="")

    returncode = process.wait()
    if returncode != 0:
        raise subprocess.CalledProcessError(returncode, train_cmd)

    print("\n" + "=" * 80)
    print("Finetune完了")
    print("=" * 80)
    print(f"終了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 実際に作成されたrunディレクトリを特定
    def resolve_actual_run_dir(project_dir: Path, name_prefix: str) -> Path:
        """name_prefixで始まるrunディレクトリの中で最新のものを返す"""
        candidates = [p for p in project_dir.glob(f"{name_prefix}*") if p.is_dir()]
        if not candidates:
            raise FileNotFoundError(f"run dir not found under {project_dir} (prefix={name_prefix})")
        return max(candidates, key=lambda p: p.stat().st_mtime)
    
    actual_results_dir = resolve_actual_run_dir(LOCAL_OUTPUT_DIR, run_name)
    results_dir = actual_results_dir
    weights_dir = results_dir / "weights"
    
    print("\n[run dir resolved]")
    print(f"  実際の保存先: {results_dir}")
    print(f"  重みファイル: {weights_dir}")
    
    # 後続セルでも使えるように環境変数にも保存
    os.environ["YOLO_RESULTS_DIR"] = str(results_dir)
    os.environ["YOLO_WEIGHTS_DIR"] = str(weights_dir)

except subprocess.CalledProcessError as e:
    print(f"\n❌ エラー: Finetune中にエラーが発生しました")
    print(f"   終了コード: {e.returncode}")
    raise

finally:
    os.chdir(original_cwd)


## セル7: 学習結果の確認


In [ ]:
# ============================================
# 学習結果の確認
# ============================================

from pathlib import Path
import os

# 学習セルで解決済みのパスを優先
if "YOLO_WEIGHTS_DIR" in os.environ:
    weights_dir = Path(os.environ["YOLO_WEIGHTS_DIR"])
    results_dir = weights_dir.parent
else:
    results_dir = LOCAL_OUTPUT_DIR / run_name
    weights_dir = results_dir / "weights"

print(f"results_dir = {results_dir}")
print(f"weights_dir = {weights_dir}")

print("=" * 80)
print("学習結果")
print("=" * 80)
print(f"結果ディレクトリ: {results_dir}")

# 重みファイルの確認
if weights_dir.exists():
    weight_files = list(weights_dir.glob('*.pt'))
    print(f"\n重みファイル数: {len(weight_files)} 件")
    
    for wf in sorted(weight_files):
        size_mb = wf.stat().st_size / (1024 * 1024)
        print(f"  {wf.name}: {size_mb:.2f} MB")
    
    # 最良モデル
    best_model = weights_dir / "best.pt"
    if best_model.exists():
        print(f"\n✓ 最良モデル: {best_model.name}")
        print(f"  サイズ: {best_model.stat().st_size / (1024 * 1024):.2f} MB")
    
    # 最終モデル
    last_model = weights_dir / "last.pt"
    if last_model.exists():
        print(f"✓ 最終モデル: {last_model.name}")
        print(f"  サイズ: {last_model.stat().st_size / (1024 * 1024):.2f} MB")
else:
    print("\n⚠️  重みファイルが見つかりません")

# 学習曲線の確認
results_csv = results_dir / "results.csv"
if results_csv.exists():
    print(f"\n学習曲線データ: {results_csv}")

# プロット画像の確認
plot_files = list(results_dir.glob("*.png"))
if len(plot_files) > 0:
    print(f"\nプロット画像数: {len(plot_files)} 件")
    for pf in plot_files:
        print(f"  {pf.name}")


## セル8: モデル重みをS3にアップロード


In [ ]:
# ============================================
# モデル重みをS3にアップロード
# ============================================

from pathlib import Path
import os
import subprocess
import time

print("=" * 80)
print("モデル重みをS3にアップロード")
print("=" * 80)

# 学習セルで解決済みのパスを優先
if "YOLO_WEIGHTS_DIR" in os.environ:
    weights_dir = Path(os.environ["YOLO_WEIGHTS_DIR"])
    results_dir = weights_dir.parent
else:
    results_dir = LOCAL_OUTPUT_DIR / run_name
    weights_dir = results_dir / "weights"

print(f"results_dir = {results_dir}")
print(f"weights_dir = {weights_dir}")

# AWS認証情報の確認
try:
    result = subprocess.run(["aws", "sts", "get-caller-identity"], 
                          capture_output=True, text=True, check=True)
    print(f"\n✓ AWS認証情報を確認")
except subprocess.CalledProcessError:
    print("\n❌ エラー: AWS認証情報が設定されていません")
    print("   セル0でAWS認証情報を設定してください")
    raise

# S3の保存先パス
actual_run_name = results_dir.name
s3_weights_prefix = f"yolo-dataset/finetune2-weights/{actual_run_name}"

# アップロードするファイルのリスト
files_to_upload = []

# 1. 最良モデル（best.pt）
best_model = weights_dir / "best.pt"
if best_model.exists():
    files_to_upload.append(("最良モデル", best_model, f"{s3_weights_prefix}/best.pt"))
else:
    print("⚠️  最良モデル（best.pt）が見つかりません")

# 2. 最終モデル（last.pt）
last_model = weights_dir / "last.pt"
if last_model.exists():
    files_to_upload.append(("最終モデル", last_model, f"{s3_weights_prefix}/last.pt"))

# 3. 学習曲線データ（results.csv）
results_csv = results_dir / "results.csv"
if results_csv.exists():
    files_to_upload.append(("学習曲線データ", results_csv, f"{s3_weights_prefix}/results.csv"))

# 4. data.yaml（学習に使用した設定）
if data_yaml_file.exists():
    files_to_upload.append(("data.yaml", data_yaml_file, f"{s3_weights_prefix}/data.yaml"))

# 5. hyp.yaml（fliplr=0.0設定）
if hyp_yaml_file.exists():
    files_to_upload.append(("hyp.yaml", hyp_yaml_file, f"{s3_weights_prefix}/hyp.yaml"))

print(f"\nアップロードするファイル数: {len(files_to_upload)} 件")
print(f"S3パス: s3://{S3_BUCKET}/{s3_weights_prefix}/")
print()

# ファイルをアップロード
uploaded = []
failed = []

for desc, local_file, s3_key in files_to_upload:
    if not local_file.exists():
        print(f"⚠️  スキップ: {desc}（ファイルが見つかりません）")
        continue
    
    s3_path = f"s3://{S3_BUCKET}/{s3_key}"
    size_mb = local_file.stat().st_size / (1024 * 1024)
    
    print(f"\n📤 {desc} をアップロード中...")
    print(f"   ファイル: {local_file.name}")
    print(f"   サイズ: {size_mb:.2f} MB")
    print(f"   S3パス: {s3_path}")
    
    max_retries = 3
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            start_time = time.time()
            result = subprocess.run(
                ["aws", "s3", "cp", str(local_file), s3_path],
                capture_output=True, text=True, check=True
            )
            elapsed = time.time() - start_time
            print(f"  ✓ 完了: {elapsed:.1f}秒")
            uploaded.append(desc)
            success = True
        except subprocess.CalledProcessError as e:
            retry_count += 1
            if retry_count < max_retries:
                print(f"  ⚠️  リトライ {retry_count}/{max_retries}...")
                time.sleep(2)
            else:
                print(f"  ❌ 失敗: {e.stderr}")
                failed.append(desc)

print(f"\n" + "=" * 80)
print("アップロード結果")
print("=" * 80)
print(f"  成功: {len(uploaded)} 件")
if uploaded:
    for name in uploaded:
        print(f"    ✓ {name}")

print(f"  失敗: {len(failed)} 件")
if failed:
    for name in failed:
        print(f"    ❌ {name}")

if len(uploaded) == len(files_to_upload):
    print(f"\n✓ 全てのファイルのアップロードが完了しました")
    print(f"  S3パス: s3://{S3_BUCKET}/{s3_weights_prefix}/")
    print(f"\n次回は以下のパスからモデルを読み込めます:")
    print(f"  最良モデル: s3://{S3_BUCKET}/{s3_weights_prefix}/best.pt")
    print(f"\n⭐ このモデルは fliplr=0.0 で学習されています（括弧の識別境界が矯正済み）")
else:
    print(f"\n⚠️  一部のファイルのアップロードに失敗しました")
    print(f"  失敗したファイルは手動でアップロードしてください")
